In [ ]:
# Titanic Data Set을 이용해서 이진분류(Logistic Regression)을
# 구현해 보아요!

# 필요한 module을 import
import numpy as np
import pandas as pd
from sklearn import linear_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Raw Data Loading
titanic = pd.read_csv('/content/drive/MyDrive/[빅데이터 과정 공유폴더]/data/titanic/train.csv')
display(titanic.head(3), titanic.shape)  # (891, 12)

# 사용하지 않는 컬럼부터 제거!
titanic.drop(['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin'],
             axis=1, inplace=True)
display(titanic.head(3), titanic.shape)  # (891, 12)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


(891, 12)

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S


(891, 7)

In [ ]:
# 데이터 전처리를 해 보아요!
gender_dict = {'male': 0,
               'female': 1 }
titanic['Sex'] = titanic['Sex'].map(gender_dict)
display(titanic.head(3), titanic.shape)  # (891, 12)


,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,0,22.0,1,0,S
1,1,1,1,38.0,1,0,C
2,1,3,1,26.0,0,0,S


(891, 7)

In [ ]:
# 가족처리
titanic['Family'] = titanic['SibSp'] + titanic['Parch']
titanic.drop(['SibSp', 'Parch'], axis=1, inplace=True)
display(titanic.head(3), toitanic.shape)  # (891, 12)

,Survived,Pclass,Sex,Age,Embarked,Family
0,0,3,0,22.0,S,1
1,1,1,1,38.0,C,1
2,1,3,1,26.0,S,0


(891, 6)

In [ ]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       714 non-null    float64
 4   Embarked  889 non-null    object 
 5   Family    891 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 41.9+ KB


In [ ]:
# Embarked 컬럼에 결측치가 2개가 있어요!
# 결측치를 찾아서 'Q'로 대체할꺼예요!
titanic['Embarked'] = titanic['Embarked'].fillna('Q')

# Embarked 컬럼의 영문자를 숫자로 바꿔줄꺼예요!
embarked_mapping = {'S': 0,
                    'C': 1,
                    'Q': 2}
titanic['Embarked'] = titanic['Embarked'].map(embarked_mapping)
display(titanic.head(3), titanic.shape)  # (891, 12)

,Survived,Pclass,Sex,Age,Embarked,Family
0,0,3,0,22.0,0,1
1,1,1,1,38.0,1,1
2,1,3,1,26.0,0,0


(891, 6)

In [ ]:
# 나이중에 NaN이 있어요!
# 처리해야 해요.. 그런데 NaN의 비중이 20%되요!
# 이런경우에는 삭제하면 당연히 좋지 않아요!
# NaN값을 적절하게 대체해서 사용할꺼예요.. 어떻게 대체하느냐에 따라
# 머신러닝의 결과가 많이 달라질 수 있어요. 그래서 조심해야 해요!
# 다른 머신러닝 기법을 이용해서 이 NaN이 어떤값이 되면 좋을지를 추측하는 방법
# 다른 방법은 전통적인 통계치를 이용하는 방법이 있고 일반적으로 많이 사용
# 평균값을 사용해요!

# 가장 간단한 방법은 나이의 평균값을 구해서 NaN을 채우는 방법이예요!
# 여자는 여자의 나이 평균을 구해서 채우고 남자는 남자의 나이 평균을 구해서 채워요!
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].mean())

In [ ]:
# 나이는 어떻게 처리하면 좋을까요?
# 그냥 실제값을 사용하는게 좋은가요? => 좋지 않을꺼 같아요!
# Binning처리를 해요! (구간을 이용해서 어떤 구간에 포함되는지를 명시)
# 8살 미만이면 0
# 8살부터 20살까지면 1 이런식으로 처리해볼꺼예요!

# 8살 미만이면 0으로 나이를 대체할꺼예요!
titanic.loc[titanic['Age'] < 8, 'Age'] = 0
titanic.loc[(titanic['Age'] >= 8) & (titanic['Age'] < 20), 'Age'] = 1
titanic.loc[(titanic['Age'] >= 20) & (titanic['Age'] < 65), 'Age'] = 2
titanic.loc[titanic['Age'] >= 65, 'Age'] = 3
display(titanic.head(3), titanic.shape)  # (891, 6)

,Survived,Pclass,Sex,Age,Embarked,Family
0,0,3,0,2.0,0,1
1,1,1,1,2.0,1,1
2,1,3,1,2.0,0,0


(891, 6)

In [ ]:
# 데이터 전처리가 끝나요!

# 전체 Training data set을 준비해 보아요!
x_data = titanic.drop('Survived', axis=1, inplace=False).values
t_data = titanic['Survived'].values.reshape(-1,1)

# training data와 test data로 분리해야 해요!
# 그런 다음 training data를 다시 training과 validation용으로 다시 나누어야 해요!
# 그런데 이런 validation 데이터를 나눠서 중간평가를 하는건 Keras가 대신 해줘요!
# 따라서 특별한 경우가 아니면 validation data는 keras를 이용해서 처리하면 되요!

In [ ]:
# Model을 만들면 되요!

keras_model = Sequential()

keras_model.add(Flatten(input_shape=(5,)))
keras_model.add(Dense(units=1,
                      activation='sigmoid'))

keras_model.compile(optimizer=Adam(learning_rate=1e-2),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

keras_model.fit(x_data,
                t_data,
                epochs=300,
                verbose=1,
                validation_split=0.2)

Epoch 1/300
23/23 [==============================] - 1s 9ms/step - loss: 1.5586 - accuracy: 0.3666 - val_loss: 1.3058 - val_accuracy: 0.2458
Epoch 2/300
23/23 [==============================] - 0s 3ms/step - loss: 1.0433 - accuracy: 0.2711 - val_loss: 0.9275 - val_accuracy: 0.2291
Epoch 3/300
23/23 [==============================] - 0s 3ms/step - loss: 0.8389 - accuracy: 0.3343 - val_loss: 0.7898 - val_accuracy: 0.4358
Epoch 4/300
23/23 [==============================] - 0s 3ms/step - loss: 0.7596 - accuracy: 0.5379 - val_loss: 0.7231 - val_accuracy: 0.6257
Epoch 5/300
23/23 [==============================] - 0s 3ms/step - loss: 0.7059 - accuracy: 0.6124 - val_loss: 0.6736 - val_accuracy: 0.6704
Epoch 6/300
23/23 [==============================] - 0s 3ms/step - loss: 0.6649 - accuracy: 0.6278 - val_loss: 0.6334 - val_accuracy: 0.6760
Epoch 7/300
23/23 [==============================] - 0s 4ms/step - loss: 0.6322 - accuracy: 0.6433 - val_loss: 0.6014 - val_accuracy: 0.6704
Epoch 8/300
2

In [ ]:
# 만들어진 모델을 이용해서 제출파일을 만들어야 해요!
# 제공된 test.csv를 이용해서 파일을 만들면 되요!

# test.csv를 이용해서 우리 모델에 대한 예측값을 얻고
# 그 값으로 gender_submission.csv 파일 형식으로 결과 파일을 만들어서
# Kaggle에 upload해서 모델의 정확도를 측정 받으시면 되요!


In [ ]:
# scikit-learn으로 학습하고 평가까지 진행해 보아요!

# scikit-learn으로 학습하고 평가까지 진행하려면 당연히
# test data가 있어야 해요!

# 전체 데이터를 training data와 test data로 분리할꺼예요!
# 여기서 말하는 전체데이터는 x_data, t_data를 의미해요
# 7:3 비율로 데이터를 나눌꺼예요!

from sklearn.model_selection import train_test_split
from sklearn import linear_model

x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(x_data,
                 t_data,
                 test_size=0.3)

sklearn_model = linear_model.LogisticRegression()
sklearn_model.fit(x_data_train, t_data_train)

sklearn_model.score(x_data_test, t_data_test)  # 0.8171641791044776

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7798507462686567

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/[빅데이터 과정 공유폴더]/data/titanic/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/[빅데이터 과정 공유폴더]/data/titanic/gender_submission.csv')

# display(test.head())
# display(submission.head())

# 테스트데이터 전처리
# 사용하지 않는 column 제거
test.drop(['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin'], axis=1, inplace=True)

# 성별 처리
gender_mapping = { 'male' : 0, 'female' : 1 }
test['Sex'] = test['Sex'].map(gender_mapping)

# 가족처리
test['Family'] = test['SibSp'] + test['Parch']
test.drop(['SibSp', 'Parch'], axis=1, inplace=True)

# Embarked 결측치 처리
test['Embarked'] = test['Embarked'].fillna('Q')

# Embarked 문자를 숫자로 변환
embarked_mapping = { 'S' : 0, 'C' : 1, 'Q' : 2 }
test['Embarked'] = test['Embarked'].map(embarked_mapping)

# Age 결측치 처리
test['Age'] = test['Age'].fillna(test['Age'].mean())

# Age Binning 처리(Numerical Value -> Categorical Value)
test.loc[test['Age'] < 8, 'Age'] = 0
test.loc[(test['Age'] >= 8) & (test['Age'] < 20), 'Age'] = 1
test.loc[(test['Age'] >= 20) & (test['Age'] < 65), 'Age'] = 2
test.loc[test['Age'] >= 65, 'Age'] = 3

# display(test)

predict = keras_model.predict(test)

submission['Survived'] = predict
submission['Survived'] = (submission['Survived'] > 0.5).astype('int')
submission.to_csv('sub.csv', index=False)

14/14 [==============================] - 0s 1ms/step


In [ ]:
# Multinomial Classification을 구현해 보아요!
# 사용하는 데이터셋은 BMI 데이터셋이예요!

# tensorflow keras와 scikit-learn을 이용해서 구현해보아요!

# 일단 필요한 모듈부터 import를 해 보아요!
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Raw Data Loading
df = pd.read_csv('/content/drive/MyDrive/[빅데이터 과정 공유폴더]/data/bmi/bmi.csv',
                 skiprows=3)
# display(df.head(), df.shape)  # (20000, 3)

# 데이터 전처리
# 1. 결측치를 찾아서 만약 존재하면 처리해야 해요!
# df.info()  # 결측치가 없네요!
# df.isnull().sum()

# 2. 이상치를 처리해야 해요! => 없어요!!
# plt.boxplot(df['label'].values)  # label에는 이상치가 없어요!
# plt.hist(df['label'].values, bins=3)
# plt.boxplot(df['height'].values)  # height에는 이상치가 없어요!
# plt.boxplot(df['weight'].values)  # weight에는 이상치가 없어요!
# plt.show()

# 3. 정규화를 해야 해요!
x_data = df[['height', 'weight']].values  # 2차원
t_data = df['label']   # 1차원
scaler = MinMaxScaler()
scaler.fit(x_data)

x_data_norm = scaler.transform(x_data)

# 4. train data와 test data를 분할해야 해요~!
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(x_data,
                 t_data,
                 test_size=0.3)


In [ ]:
# Scikit Learn 구현
sklearn_model = linear_model.LogisticRegression()
sklearn_model.fit(x_data_train,
                  t_data_train)

sklearn_model_result = sklearn_model.score(x_data_test,
                                           t_data_test)
print('sklearn model의 accuracy : {}'.format(sklearn_model_result))
# 0.9823
# 187, 82
print(sklearn_model.predict_proba([[187, 82]]))
# [[4.55458751e-17 9.99979818e-01 2.01819517e-05]]

sklearn model의 accuracy : 0.9825
[[2.27635505e-22 9.99678371e-01 3.21629010e-04]]


In [ ]:
# Tensorflow Keras 구현
keras_model = Sequential()

keras_model.add(Flatten(input_shape=(2,)))
keras_model.add(Dense(units=3,
                      activation='softmax'))
keras_model.compile(optimizer=Adam(learning_rate=1e-2),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

# 정규화된 데이터를 사용해야 해요!
x_data_norm_train, x_data_norm_test, t_data_train, t_data_test = \
train_test_split(x_data_norm,
                 t_data,
                 test_size=0.3)
keras_model.fit(x_data_norm_train,
                t_data_train,
                epochs=200,
                verbose=1,
                validation_split=0.3)

Epoch 1/200
307/307 [==============================] - 2s 4ms/step - loss: 0.8394 - accuracy: 0.7247 - val_loss: 0.6663 - val_accuracy: 0.8176
Epoch 2/200
307/307 [==============================] - 1s 2ms/step - loss: 0.5844 - accuracy: 0.8558 - val_loss: 0.5232 - val_accuracy: 0.8593
Epoch 3/200
307/307 [==============================] - 1s 2ms/step - loss: 0.4795 - accuracy: 0.8931 - val_loss: 0.4445 - val_accuracy: 0.9048
Epoch 4/200
307/307 [==============================] - 1s 3ms/step - loss: 0.4157 - accuracy: 0.9209 - val_loss: 0.3920 - val_accuracy: 0.9352
Epoch 5/200
307/307 [==============================] - 1s 3ms/step - loss: 0.3711 - accuracy: 0.9403 - val_loss: 0.3536 - val_accuracy: 0.9457
Epoch 6/200
307/307 [==============================] - 1s 3ms/step - loss: 0.3377 - accuracy: 0.9515 - val_loss: 0.3235 - val_accuracy: 0.9590
Epoch 7/200
307/307 [==============================] - 1s 4ms/step - loss: 0.3105 - accuracy: 0.9641 - val_loss: 0.2995 - val_accuracy: 0.9652

In [ ]:
# 학습이 다 되면!!
# 평가를 해야 해요!
print(keras_model.evaluate(x_data_norm_test, t_data_test))
# [0.060929279774427414, 0.9858333468437195]

# Prediction
my_state = np.array([[170, 58]])
my_state_norm = scaler.transform(my_state)
print(keras_model.predict(my_state_norm))
# [[7.1084266e-08 9.3678677e-01 6.3213095e-02]]

188/188 [==============================] - 0s 2ms/step - loss: 0.0583 - accuracy: 0.9860
[0.05830681696534157, 0.9860000014305115]
1/1 [==============================] - 0s 65ms/step
[[2.4757301e-02 9.7524059e-01 2.0389066e-06]]
